# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userd'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [6]:
##Keyspace creation
try:
    session.execute("create keyspace if not exists cassandraproject with replication \
    = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
##Setting keyspace
try:
    session.set_keyspace('cassandraproject')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
##Create verifying table
query = "create table if not exists songsession"
query = query + "(session_id int, item_in_session int,userid int, artist_name text, song text,\
                song_length double, first_name text, last_name text,\
                 primary key ((session_id,item_in_session), userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
##Insert data into verifying table
##Open data set
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Insert statement description
        query = "insert into songsession(session_id, item_in_session, userid, artist_name, \
                song, song_length, first_name, last_name)"
        query = query + " values (%s, %s, %s, %s, %s, %s, %s, %s)"
        ##Column data assignment
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[0], line[9], float(line[5]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
##Data check select statement
query = "select artist_name, song, song_length from songsession where item_in_session = 4 and session_id = 338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
##Query 1 create table
query = "drop table if exists query1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "create table if not exists query1"
query = query + "(session_id int, item_in_session int,userid int, artist_name text, song text,\
                song_length double, first_name text, last_name text,\
                 primary key ((session_id,item_in_session), userid))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
##Insert data into verifying table
##Open data set
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##Insert statement description
        query = "insert into query1(session_id, item_in_session, userid, artist_name, \
                song, song_length, first_name, last_name)"
        query = query + " values (%s, %s, %s, %s, %s, %s, %s, %s)"
        ##Column data assignment
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[0], line[9], float(line[5]), line[1], line[4]))

In [13]:
##Query 1 select
query = "select artist_name, song, song_length from query1 where item_in_session = 4 and session_id = 338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


In [14]:
##Query 2 create table
query = "drop table if exists query2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "create table if not exists query2"
query = query + "(userid int, session_id int, item_in_session int, artist_name text, song text, \
                first_name text, last_name text, primary key (userid, session_id));"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [15]:
##Insert data into verifying table
##Open data set
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ##Insert statement description
        query = "insert into query2 (userid, session_id, item_in_session, artist_name, song, first_name, last_name)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s) IF NOT EXISTS"
        ##Column data assignment
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [93]:
##Query 2 select
query = "select userid, session_id, item_in_session, artist_name, song, first_name, last_name from query2 where userid = 10 and session_id =182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.userid, row.session_id, row.item_in_session, row.artist_name, row.song, row.first_name, row.last_name)

10 182 0 Down To The Bone Keep On Keepin' On Sylvie Cruz


In [94]:
##Query 3 create table
query = "drop table if exists query3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "create table if not exists query3"
query = query + "(song varchar, userid int, first_name varchar, \
                last_name varchar, PRIMARY KEY (song, userid));"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [95]:
##Insert data into verifying table
##Open data set
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    ##Insert statement description
        query = "insert into query3 (song, userid, first_name, last_name)"
        query = query + "values (%s, %s, %s, %s) IF NOT EXISTS"
        ##Column data assignment
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [96]:
##Query 3 select
query = "select first_name, last_name from query3 where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [97]:
query = "drop table songsession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table query1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table query2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table query3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()